In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 84.8 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-11-07 13:45:49.024623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762523149.263461      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762523149.332188      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [12]:
from langchain.llms.base import LLM
from typing import Any

class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text(prompt, max_length=500)
    
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Chain 1: Generate app idea
idea_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Come up with a creative app idea related to {topic}. Keep it short."
)
idea_chain = LLMChain(llm=llm, prompt=idea_prompt)

# Chain 2: Extract features
feature_prompt = PromptTemplate(
    input_variables=["app_idea"],
    template="Given this app idea: {app_idea}, list its top 3 features in a bullet list."
)
feature_chain = LLMChain(llm=llm, prompt=feature_prompt)

# Chain 3: Create tagline
tagline_prompt = PromptTemplate(
    input_variables=["app_idea", "features"],
    template=(
        "Based on the app idea: {app_idea} and these features:\n{features},\n"
        "write a catchy tagline (1 sentence)."
    )
)

tagline_chain = LLMChain(llm=llm, prompt=tagline_prompt)

In [ ]:
def run_all(topic: str):
    print(f"Topic: {topic}")
    
    app_idea = idea_chain.run(topic)
    print(f"\nApp Idea:\n{app_idea.strip()}")
    
    features = feature_chain.run(app_idea)
    print(f"\nFeatures:\n{features.strip()}")
    
    tagline = tagline_chain.run({
        "app_idea": app_idea,
        "features": features
    })
    print(f"\nTagline:\n{tagline.strip()}")

In [17]:
run_all("Football")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Topic: Football


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



App Idea:
Come up with a creative app idea related to Football. Keep it short. It's called "Captain's Pick."

"Captain's Pick" is an app that allows users to select their favorite football players as their 'captains' and track their performance throughout the season. Users can also compare their captains' stats with other users' captains, creating a fun and engaging competitive environment. The app will provide real-time updates, personalized news feeds, and exclusive interviews with the captains themselves. Additionally, users can create and join leagues with friends, adding a social aspect to the app.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Features:
Given this app idea: Come up with a creative app idea related to Football. Keep it short. It's called "Captain's Pick."

"Captain's Pick" is an app that allows users to select their favorite football players as their 'captains' and track their performance throughout the season. Users can also compare their captains' stats with other users' captains, creating a fun and engaging competitive environment. The app will provide real-time updates, personalized news feeds, and exclusive interviews with the captains themselves. Additionally, users can create and join leagues with friends, adding a social aspect to the app., list its top 3 features in a bullet list. Here are the top 3 features of "Captain's Pick":

- **Real-time Player Performance Tracking**: Users can monitor their captains' stats and progress in real-time, keeping them engaged and informed throughout the season.
- **Social League Creation**: Users can create and join leagues with friends, fostering competition and c